First we define methods to collect our data

In [30]:
from datetime import datetime, timezone, timedelta
import os
import pandas as pd # data processing
import bitmex
import math
from tqdm.notebook import trange, tqdm # progress bar
import time
# import warnings  
# warnings.filterwarnings('ignore')

import argparse, json, math, time, pdb, os, copy, uuid
from pprint import pprint
from box import Box
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from sqlalchemy.sql import text
# import tensorforce.agents as agents
import tensorforce
# from tensorforce.exception import TensorforceError
from tensorforce.agents.policy_agent import ProximalPolicyOptimization
from tensorforce.agents.ppo import ProximalPolicyOptimization
#from tensorforce.core.networks import layer as TForceLayers
#from tensorforce.core.networks.network import LayeredNetwork

from sqlalchemy.dialects import mysql as psql
from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll.base import scope

from btc_env import BitcoinEnv
import utils
from data import data

test_mode = False
key = '1rw8DDZoBd3xGXOmo48lFfQv' if test_mode else '73rdj2LQf63ANuFczBOlINsS'
secret = 'ODoX-aeLDeLSPWidAtqNpnN70eWDKLgfdpdIUL6_fifPZhEu' if test_mode else 'TmIdFVEN-Muni_TAKpA0zghts7EHv5ytq5vZfrPQXRxM01x7'
client = bitmex.bitmex(test=test_mode, api_key=key, api_secret=secret)
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1h": 240, "1d": 1440}
batch_size = 750

def get_all_bitmex(symbol, kline_size, save = False, start_time = None):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    oldest_point = start_time if start_time is not None else oldest_point
    print(oldest_point)
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
#     data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def minutes_of_new_data(symbol, kline_size, data, source):
    print(kline_size)
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

ImportError: cannot import name 'TensorforceError'

In [3]:
parser = argparse.ArgumentParser()
utils.add_common_args(parser)
args = parser.parse_args()

NameError: name 'argparse' is not defined